In [15]:
#Importando bibliotecas que serão utilizadas no código
from numpy import zeros, ones, expand_dims, asarray
from numpy.random import randn, randint
from keras.datasets import fashion_mnist
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import Conv2D, Conv2DTranspose, Concatenate
from keras.layers import LeakyReLU, Dropout, Embedding
from keras.layers import BatchNormalization, Activation
from keras import initializers
from keras.initializers import RandomNormal
from keras.optimizers import Adam, RMSprop, SGD
from matplotlib import pyplot
import numpy as np
from math import sqrt

In [ ]:
#Carregando dataset 
# carregando apenas as características dos dados de treinamento, pois não precisamos dos rótulos.
# Em seguida, estamos dividindo cada valor de pixel por 127,5 e subtraindo 1 para ter valores de pixel na faixa de -1 a 1. Por fim, a forma de X_train é (60000, 28, 28, 1).

(X_train, _), (_, _) = fashion_mnist.load_data()
X_train = X_train.astype(np.float32) / 127.5 - 1
X_train = np.expand_dims(X_train, axis=3)
print(X_train.shape)

In [17]:
#Algumas funções que são necessarias
#  usando a função abaixo para gerar pontos latentes com a forma n_samplesxlatent_dim (100 no nosso caso).
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)  
    z_input = x_input.reshape(n_samples, latent_dim)
    return z_input

In [18]:
# A função abaixo ajuda a gerar n amostras reais com o rótulo 1, ou seja, uma imagem real.

def generate_real_samples(X_train, n_samples):
    ix = randint(0, X_train.shape[0], n_samples) 
    X = X_train[ix]  
    y = ones((n_samples, 1)) 
    return X, y

In [19]:
# A função acima ajuda a gerar (n) amostras falsas usando o gerador com o rótulo 0, ou seja, uma imagem falsa.

def generate_fake_samples(generator, latent_dim, n_samples):
    z_input = generate_latent_points(latent_dim, n_samples)
    images = generator.predict(z_input)  
    y = zeros((n_samples, 1))
    return images, y

In [20]:
# Esta função ajuda a resumir o desempenho. Isso inclui a geração de uma amostra falsa, a plotagem dela e, finalmente, o salvamento do modelo.

def summarize_performance(step, g_model, latent_dim, n_samples=100):
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    X = (X + 1) / 2.0
    for i in range(100):
        pyplot.subplot(10, 10, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
    filename2 = 'model_%04d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s' % (filename2))

In [21]:
# A função abaixo ajuda a plotar os resultados. usada para plotar as imagens geradas pelo Gerador em estágios posteriores.

def save_plot(examples, n_examples):
    for i in range(n_examples):
        pyplot.subplot(sqrt(n_examples), sqrt(n_examples), 1 + i)
        pyplot.axis('off')
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    pyplot.show()

The above function helps us to plot the results. We’ll use this to plot the generated images by the Generator in later stages.



In [22]:
# Construindo o modelo

# discriminator = define_discriminator() utilizando algumas camadas Dense, Flatten e Dropout com a função de ativação leaky relu nas camadas ocultas e sigmoid na camada final.
# Utilizando o otimizador adam e a função de perda binary cross-entropy, pois a tarefa do discriminador é realizar a classificação binária.

def define_discriminator(in_shape=(28, 28, 1)):
    init = RandomNormal(stddev=0.02)  
    in_image = Input(shape=in_shape)
    fe = Flatten()(in_image)
    fe = Dense(1024)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.3)(fe)
    fe = Dense(512)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.3)(fe)
    fe = Dense(256)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.3)(fe)
    out = Dense(1, activation='sigmoid')(fe)
    model = Model(in_image, out)
    opt = Adam(lr=0.0002, beta_1=0.5) 
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [23]:
# generator = define_generator(100)

# utilizando algumas camadas Dense para definir o modelo do gerador, novamente com a função de ativação Leaky ReLU nas camadas ocultas e (tanh) na camada final.
# As imagens geradas G(z) terão o formato 28x28x1.

def define_generator(latent_dim): 
    init = RandomNormal(stddev=0.02)
    in_lat = Input(shape=(latent_dim,)) 
    gen = Dense(256, kernel_initializer=init)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Dense(512, kernel_initializer=init)(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Dense(1024, kernel_initializer=init)(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Dense(28 * 28 * 1, kernel_initializer=init)(gen)
    out_layer = Activation('tanh')(gen)
    out_layer = Reshape((28, 28, 1))(gen)
    model = Model(in_lat, out_layer)
    return model

In [24]:
# gan_model = define_gan(generator, discriminator) congelando o discriminador, fornecendo z como entrada e D(G(z)) como saída para o nosso modelo.
#  utilizando o otimizador adam e a função de perda binary cross-entropy.

def define_gan(g_model, d_model):
    d_model.trainable = False
    gan_output = d_model(g_model.output)
    model = Model(g_model.input, gan_output)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [28]:
#Treinando o modelo

# Esta função nos ajuda a treinar o gerador e o discriminador. 
# Para treinar o discriminador, ela primeiro gera amostras reais, atualiza os pesos do discriminador, gera amostras falsas e depois atualiza novamente os pesos do discriminador. 
# Para treinar o gerador, ela primeiro gera pontos latentes, gera rótulos como 1 para enganar o discriminador e, em seguida, atualiza os pesos do gerador. 
# Por fim, a função resume o desempenho do modelo após algumas etapas.

def train(g_model, d_model, gan_model, X_train, latent_dim, n_epochs=100, n_batch=64):
    bat_per_epo = int(X_train.shape[0] / n_batch)
    n_steps = bat_per_epo * n_epochs
    for i in range(n_steps):
        X_real, y_real = generate_real_samples(X_train, n_batch)
        d_loss_r, d_acc_r = d_model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_batch)
        d_loss_f, d_acc_f = d_model.train_on_batch(X_fake, y_fake)
        z_input = generate_latent_points(latent_dim, n_batch) 
        y_gan = ones((n_batch, 1)) 
        g_loss, g_acc = gan_model.train_on_batch(z_input, y_gan)
        print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_loss_r,d_acc_r, d_loss_f,d_acc_f, g_loss,g_acc))
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, latent_dim)

This function helps us to train the generator and the discriminator. To train the Discriminator, it first generates real samples, updates the discriminator’s weights, generates fake samples, and then updates the discriminator’s weights again. To train the Generator, it first generates latent points, generates labels as 1 to fool the discriminator, and then updates the generator’s weights. Finally, the function summarizes the performance of the model after some steps.

In [ ]:
# Não consegui rodar o código a partir daqui 

# Aqui o código está chamando a função de treinamento com 100 amostras aleatórias, 20 épocas e 64 como tamanho do lote (batch size).

latent_dim = 100
train(generator, discriminator, gan_model, X_train, latent_dim, n_epochs=20, n_batch=64)

latent_dim = 100
train(generator, discriminator, gan_model, X_train, latent_dim, n_epochs=20, n_batch=64)

In [ ]:
# Aqui o código está carregando o modelo salvo mais recente, gerando pontos latentes, usando o modelo carregado para previsão e plotando os resultados.


model = load_model('model_18740.h5')
latent_dim = 100
n_examples = 100
latent_points = generate_latent_points(latent_dim, n_examples)
X  = model.predict(latent_points)
X = (X + 1) / 2.0
save_plot(X, n_examples)